# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [8]:
# import libraries
import nltk
import re
nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sqlalchemy import create_engine
import pandas as pd
from itertools import chain

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
# load data from database
engine = create_engine('sqlite:///Anand_ETL_Pipeline_16022020.db')
df = pd.read_sql_table('Cyclone',engine)
X = df.message.values
y = df[['related',  'request',  'offer',  'aid_related',  'medical_help',  'medical_products',  'search_and_rescue',  'security', 'military',  'child_alone',  'water',  'food',  'shelter',  'clothing',  'money',  'missing_people',  'refugees',  'death',  'other_aid',  'infrastructure_related',  'transport',  'buildings',  'electricity',  'tools',  'hospitals',  'shops',  'aid_centers',  'other_infrastructure',  'weather_related',  'floods',  'storm',  'fire',  'earthquake',  'cold',  'other_weather',  'direct_report']].values

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    
    # Normalize text and Tokenize text
    words = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()).split()

    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    #print(words)

    # Reduce words to their root form
    lemmed_before = [WordNetLemmatizer().lemmatize(w) for w in words]
    #print(lemmed_before)

    # Lemmatize verbs by specifying pos
    lemmed_after = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed_before]
    #print(lemmed_after)
    
    return lemmed_after


token = lambda x: tokenize(x)


X = [token(df.message[x]) for x in range(len(df['message']))]
#Y =
#X = list(chain(*X))



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
# build pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=token)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
#Spliting into Training and Test data sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train classifier
pipeline.fit(X_train,y_train)



AttributeError: 'list' object has no attribute 'lower'

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X = [token(df.message[x]) for x in range(len(df['message']))]

y = df[['related',  'request',  'offer',  'aid_related',  'medical_help',  'medical_products',  'search_and_rescue',  'security', 'military',  'child_alone',  'water',  'food',  'shelter',  'clothing',  'money',  'missing_people',  'refugees',  'death',  'other_aid',  'infrastructure_related',  'transport',  'buildings',  'electricity',  'tools',  'hospitals',  'shops',  'aid_centers',  'other_infrastructure',  'weather_related',  'floods',  'storm',  'fire',  'earthquake',  'cold',  'other_weather',  'direct_report']].values

test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, y, test_size=test_size, random_state=seed)
model = LogisticRegression()
model.fit(X_train, Y_train)
predicted = model.predict(X_test)
report = classification_report(Y_test, predicted)
print(report)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
#improve the model with grid search
pipeline = Pipeline([('scaler', StandardScaler()),('clf', SVC())])

parameters = {
    'scaler__with_mean': [True, False],
    'clf__kernel': ['linear', 'rbf'],
    'clf__C':[1, 10]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!



In [ ]:
### Test your model

from sklearn.preprocessing import label_binarize

# Use label_binarize to be multi-label like settings
Y = label_binarize(y, classes=[0, 1, 2])
n_classes = Y.shape[1]

# Split into training and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.5,
                                                    random_state=random_state)

# We use OneVsRestClassifier for multi-label prediction
from sklearn.multiclass import OneVsRestClassifier

# Run classifier
classifier = OneVsRestClassifier(svm.LinearSVC(random_state=random_state))
classifier.fit(X_train, Y_train)
y_score = classifier.decision_function(X_test)


from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

# For each class
precision = dict()
recall = dict()
average_precision = dict()
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(Y_test[:, i],
                                                        y_score[:, i])
    average_precision[i] = average_precision_score(Y_test[:, i], y_score[:, i])

# A "micro-average": quantifying score on all classes jointly
precision["micro"], recall["micro"], _ = precision_recall_curve(Y_test.ravel(),
    y_score.ravel())
average_precision["micro"] = average_precision_score(Y_test, y_score,
                                                     average="micro")
print('Average precision score, micro-averaged over all classes: {0:0.2f}'
      .format(average_precision["micro"]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
### Export your model as a pickle file
import pickle

# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.